In [32]:
#importamos las librerias que se van a usar
import pandas as pd

# Leer un archivo Excel y seleccionar una hoja en específico
df = pd.read_excel("DataSet_1_1.xlsx", sheet_name="Sheet1")
df_IDHM = pd.read_excel("IDHM.xlsx", sheet_name="Base IDHM")

# Mostrar las primeras filas
print(df.head())
print(df_IDHM.head())

     Id SEXO  EDAD Departamento       Municipio    Escolaridad  \
0  I001    M    34        Petén     Desconocido  Diversificado   
1  I002    M    29   Chiquimula     Desconocido   Sin estudios   
2  I003    M    19    Guatemala     Villa Nueva  Diversificado   
3  I004    M    19       Izabal  Puerto Barrios        Básicos   
4  I005    M    29       Quiché     Desconocido       Primaria   

      NIVEL ESCOLARIDAD SABE INGLES (SI/NO) Nivel de Ingles  \
0         Diversificado                  No             NaN   
1          Sin estudios                  No             NaN   
2         Diversificado                  No             NaN   
3  Secundaria / Básicos                  No             NaN   
4              Primaria                  Si      Intermedio   

                                              OFICIO  Año Experiencia  \
0                                             pintor              4.0   
1                               campo cortando fruta             10.0   
2    

In [33]:
#Ponemos los nombres iguales en las columnas
df_IDHM = df_IDHM.rename(columns={
    "DEPARTAMENTO": "Departamento",
    "MUNICIPIO": "Municipio"
})


In [34]:
#Se va normalizar los catalogos de Departamento y municipio
import unidecode  # para quitar tildes y caracteres especiales

# Función para normalizar texto
def normalize_text(s):
    if pd.isna(s):
        return ''
    s = str(s).lower()              # convertir a minúsculas
    s = unidecode.unidecode(s)      # quitar tildes
    s = s.replace('.', '')          # quitar puntos si los hubiera
    return s

# Aplicar la funcion a las columnas
df['Departamento'] = df['Departamento'].apply(normalize_text)
df['Municipio'] = df['Municipio'].apply(normalize_text)

df_IDHM['Departamento'] = df_IDHM['Departamento'].apply(normalize_text)
df_IDHM['Municipio'] = df_IDHM['Municipio'].apply(normalize_text)

In [35]:
#Hacemos un left join para el indicador de IDHM
df = df.merge(
    df_IDHM[['Departamento', 'Municipio', 'IDHM 2002']],
    on=['Departamento', 'Municipio'],
    how='left'  
)

# Revisar valores faltantes
print(df[['Departamento','Municipio','IDHM 2002']].isna().sum())

Departamento      0
Municipio         0
IDHM 2002       294
dtype: int64


In [36]:
print(df[['Departamento','Municipio','IDHM 2002']].head(10))
print(df[['IDHM 2002']].describe())

     Departamento       Municipio  IDHM 2002
0           peten     desconocido        NaN
1      chiquimula     desconocido        NaN
2       guatemala     villa nueva   0.687794
3          izabal  puerto barrios   0.620433
4          quiche     desconocido        NaN
5       escuintla     desconocido        NaN
6          quiche          chajul   0.403506
7         jutiapa      jalpatagua   0.562542
8    baja verapaz     desconocido        NaN
9  quetzaltenango     desconocido        NaN
        IDHM 2002
count  349.000000
mean     0.510694
std      0.096734
min      0.344029
25%      0.441346
50%      0.505444
75%      0.564466
max      0.748675


In [37]:
# Promedio de IDHM por departamento ya que se desconoce el municipio
promedios_IDHM = df_IDHM.groupby('Departamento')[['IDHM 2002']].mean().reset_index()

# Merge para traer promedio
df = df.merge(promedios_IDHM, on='Departamento', suffixes=('', '_prom'))

# Rellenar NaN con promedio departamental
df['IDHM 2002'] = df['IDHM 2002'].fillna(df['IDHM 2002_prom'])

# Eliminar columnas auxiliares
df = df.drop(columns=['IDHM 2002_prom'])

print(df[['Departamento','Municipio','IDHM 2002']].head(10))
print(df[['IDHM 2002']].describe())

     Departamento       Municipio  IDHM 2002
0           peten     desconocido   0.507598
1      chiquimula     desconocido   0.493567
2       guatemala     villa nueva   0.687794
3          izabal  puerto barrios   0.620433
4          quiche     desconocido   0.434368
5       escuintla     desconocido   0.550541
6          quiche          chajul   0.403506
7         jutiapa      jalpatagua   0.562542
8    baja verapaz     desconocido   0.505567
9  quetzaltenango     desconocido   0.542240
        IDHM 2002
count  617.000000
mean     0.500083
std      0.079052
min      0.344029
25%      0.449358
50%      0.504953
75%      0.538276
max      0.748675


In [38]:
# Revisar valores faltantes
print(df[['Departamento','Municipio','IDHM 2002']].isna().sum())

Departamento    0
Municipio       0
IDHM 2002       0
dtype: int64


In [39]:
#se va leer el otro dataset que tiene los indicadores IPM
df_IPM = pd.read_excel("IDHM.xlsx", sheet_name="Base IPM")

#Ponemos los nombres iguales en las columnas
df_IPM = df_IPM.rename(columns={
    "DEPARTAMENTO": "Departamento",
    "MUNICIPIO": "Municipio"
})

# Aplicar la funcion de normalizar a las columnas
df_IPM['Departamento'] = df_IPM['Departamento'].apply(normalize_text)
df_IPM['Municipio'] = df_IPM['Municipio'].apply(normalize_text)

#Hacemos el left join para el indicador de IPM
df = df.merge(
    df_IPM[['Departamento', 'Municipio', 'IPM EDUCACION','IPM EMPLEO','IPM VIVIENDA','IPM SERV. BASICOS','IPM']],
    on=['Departamento', 'Municipio'],
    how='left'  
)

# Revisar valores faltantes
print(df[['Departamento','Municipio','IPM EDUCACION','IPM EMPLEO','IPM VIVIENDA','IPM SERV. BASICOS','IPM']].isna().sum())

print(df[['Departamento','Municipio','IPM EDUCACION','IPM EMPLEO','IPM VIVIENDA','IPM SERV. BASICOS','IPM']].head(10))
print(df[['IDHM 2002']].describe())

Departamento           0
Municipio              0
IPM EDUCACION        268
IPM EMPLEO           268
IPM VIVIENDA         268
IPM SERV. BASICOS    268
IPM                  268
dtype: int64
     Departamento       Municipio  IPM EDUCACION  IPM EMPLEO  IPM VIVIENDA  \
0           peten     desconocido            NaN         NaN           NaN   
1      chiquimula     desconocido            NaN         NaN           NaN   
2       guatemala     villa nueva          0.175    0.129283      0.093611   
3          izabal  puerto barrios          0.226    0.186063      0.104220   
4          quiche     desconocido            NaN         NaN           NaN   
5       escuintla     desconocido            NaN         NaN           NaN   
6          quiche          chajul          0.498    0.440530      0.681797   
7         jutiapa      jalpatagua          0.300    0.253702      0.252856   
8    baja verapaz     desconocido            NaN         NaN           NaN   
9  quetzaltenango     desconocid

In [40]:
# Promedio de IPM por departamento ya que se desconoce el municipio
promedios_IPM = df_IPM.groupby('Departamento')[['IPM EDUCACION','IPM EMPLEO','IPM VIVIENDA','IPM SERV. BASICOS','IPM']].mean().reset_index()

# Merge para traer promedio
df = df.merge(promedios_IPM, on='Departamento', suffixes=('', '_prom'))

# Rellenar NaN con promedio departamental
df['IPM EDUCACION'] = df['IPM EDUCACION'].combine_first(df['IPM EDUCACION_prom'])
df['IPM EMPLEO'] = df['IPM EMPLEO'].combine_first(df['IPM EMPLEO_prom'])
df['IPM VIVIENDA'] = df['IPM VIVIENDA'].combine_first(df['IPM VIVIENDA_prom'])
df['IPM SERV. BASICOS'] = df['IPM SERV. BASICOS'].combine_first(df['IPM SERV. BASICOS_prom'])
df['IPM'] = df['IPM'].combine_first(df['IPM_prom'])

# Eliminar columnas auxiliares
df = df.drop(columns=['IPM EDUCACION_prom', 'IPM EMPLEO_prom', 'IPM VIVIENDA_prom', 'IPM SERV. BASICOS_prom', 'IPM_prom'])

print(df[['Departamento','Municipio','IPM EDUCACION','IPM EMPLEO','IPM VIVIENDA','IPM SERV. BASICOS','IPM']].head(10))

     Departamento       Municipio  IPM EDUCACION  IPM EMPLEO  IPM VIVIENDA  \
0           peten     desconocido       0.363857    0.301729      0.302765   
1      chiquimula     desconocido       0.372727    0.273340      0.427447   
2       guatemala     villa nueva       0.175000    0.129283      0.093611   
3          izabal  puerto barrios       0.226000    0.186063      0.104220   
4          quiche     desconocido       0.461810    0.323532      0.517747   
5       escuintla     desconocido       0.303929    0.213027      0.200929   
6          quiche          chajul       0.498000    0.440530      0.681797   
7         jutiapa      jalpatagua       0.300000    0.253702      0.252856   
8    baja verapaz     desconocido       0.350875    0.330697      0.390122   
9  quetzaltenango     desconocido       0.344833    0.264378      0.184994   

   IPM SERV. BASICOS       IPM  
0           0.380666  0.400563  
1           0.289789  0.407155  
2           0.092642  0.122477  
3        

In [43]:
#Se tiene los siguientes valores segun la Organizacion Internacional de Trabajo ILO Stats
df_ilo = pd.DataFrame({
    'SEXO': ['M','M','M','M','M','M',
                  'F','F','F','F','F','F'],
    'age_range': ['15-24','25-34','35-44','45-54','55-64','65+',
                       '15-24','25-34','35-44','45-54','55-64','65+'],
    'employment_rate': [75.472,95.856,96.128,95.67,89.574,61.049,
                  41.288,58.494,65.046,59.98,47.935,24.758]
})

# Creamos una funcion para mapear las edades
def edad_rango(age):
    if age <= 24:
        return '15-24'
    elif age <= 34:
        return '25-34'
    elif age <= 44:
        return '35-44'
    elif age <= 54:
        return '45-54'
    elif age <= 64:
        return '55-64'
    else:
        return '65+'

# Creamos columna con rango de edad
df['age_range'] = df['EDAD'].apply(edad_rango)

# Unimos los valores de tasa de empleo segun el sexo
df = df.merge(df_ilo, on=['SEXO','age_range'], how='left')

# Revisar resultado
print(df[['EDAD','SEXO','age_range','employment_rate']].head(10))

   EDAD SEXO age_range  employment_rate
0    34    M     25-34           95.856
1    29    M     25-34           95.856
2    19    M     15-24           75.472
3    19    M     15-24           75.472
4    29    M     25-34           95.856
5    32    F     25-34           58.494
6    43    M     35-44           96.128
7    35    M     35-44           96.128
8    27    M     25-34           95.856
9    22    F     15-24           41.288


In [44]:
# Se tiene la siguiente informacion de tasa de desempleo segun el ILO Stats
df_tdes = pd.DataFrame({
    'SEXO': ['M','M','F','F'],
    'age_range2': ['15-24','25+','15-24','25+'],
    'unemployment_rate': [4.141,1.108,4.66,2.382]
})

#Crear una funcion para mapear las edades
def edad_desempleo(age):
    if age <= 24:
        return '15-24'
    else:
        return '25+'  # Todos los mayores de 24

# Creamos columna con rango de edad2
df['age_range2'] = df['EDAD'].apply(edad_desempleo)

# Unimos los valores de tasa de desempleo segun el sexo
df = df.merge(df_tdes, on=['SEXO','age_range2'], how='left')

# Revisar resultado
print(df[['EDAD','SEXO','age_range2','unemployment_rate']].head(10))

   EDAD SEXO age_range2  unemployment_rate
0    34    M        25+              1.108
1    29    M        25+              1.108
2    19    M      15-24              4.141
3    19    M      15-24              4.141
4    29    M        25+              1.108
5    32    F        25+              2.382
6    43    M        25+              1.108
7    35    M        25+              1.108
8    27    M        25+              1.108
9    22    F      15-24              4.660


In [45]:
#Se exporta a Excel el dataset final con las variables independientes
df.to_excel("DataSet_2.xlsx", index=False)